In [29]:
import json
import glob
import pandas as pd

In [30]:
# Read json results file
file = '../../astla-data/dart-preposttest/novo-asr-results/novo-asr-results-1-1287.json'

with open(file, 'r') as f:
  data = json.load(f)

# print(data[0]['result']['task_uuid'])
# print(data[0]['result']['recognized'])
# print(data[0]['result']['words'][0])
# print(data[0]['result']['words'][1]['label'])
# print(data[0]['result']['words'][1]['confidence'])
# print(data[0]['result']['words'][1]['phones'][1]['label'])
# print(data[0]['result']['words'][1]['phones'][1]['confidence'])
# print(data[0]['result']['words'][1]['phones'][1]['end']/1000)
# print(data[0]['result']['words'][1]['phones'][1]['begin']/1000)
# print(data[0]['result']['words'][1]['phones'][1]['recognized'])
# print(data[0]['result']['words'][1]['phones'][1]['recognized']['label'])
# print(data[0]['result']['words'][1]['phones'][1]['recognized']['confidence']['prob'])
# print(data[0]['result']['words'][1]['phones'][1]['confidence']['prob'])
# print(data[0]['meta']['filename'])
# print(data[0]['meta']['speaker'])

In [31]:
audioFiles = [audioFile.split('/')[-1] for audioFile in glob.glob("/vol/tensusers5/wharmsen/astla-data/dart-preposttest/slate-data/3-audio/*.mp3")]
audioFiles

['09eae7df-db7c-491b-b7c4-a7c9fcf62f47.mp3',
 '74be41e8-d940-469f-8ebf-feb90250f9e8.mp3',
 '97549707-f715-4fe5-926c-b272b388e368.mp3',
 'aa809d8a-45d1-4cc0-a2c2-b40066d2849e.mp3',
 'd52f2fbc-588a-4002-8c02-772dabdbfc16.mp3',
 'e5920085-8c1b-415c-b25d-24b5e5a4a2bf.mp3']

In [32]:
# audioFiles.append('ffdc492f-82f3-424f-98c1-c682a32b16fd.mp3')

In [33]:
slate_dict = {}
for jsonItem in data:
    audio = jsonItem['meta']['filename']
    if(audio in audioFiles):
        slate_dict[audio] = jsonItem

print(len(slate_dict))

6


In [35]:
outputList = []
for key in slate_dict.keys():
    words = slate_dict[key]['result']['words']
    for word in words:
        label = word['label']['raw']
        if(label not in ['<s>', '<sil>', '</s>']):
            confidence = word['confidence']['prob']
            novo_acc = 1 if confidence > 47 else 0
            begin = word['begin']
            end = word['end']
            target_phones = " ".join([phone['label'] + '('+ str(phone['confidence']['prob']) +')'  for phone in word['phones']])
            rec_phones = " ".join([phone['recognized']['label'] + '('+ str(phone['recognized']['confidence']['prob']) +')'  for phone in word['phones']])
            output = [key.replace(".mp3", "_"+label), novo_acc, confidence, begin, end, target_phones, rec_phones]
            outputList.append(output)

novo_df = pd.DataFrame(outputList, columns=['novo_word', 'novo_acc', 'novo_conf', 'novo_begin', 'novo_end', 'novo_target_phones', 'novo_rec_phones'])
novo_df.to_csv('../../astla-data/dart-preposttest/slate-data/5-asr-analysis-results/novo-word-assessments.csv')